# Beaver Triples

Author: 
- Carlos Salgado - [email](mailto:csalgado@uwo.ca) - [linkedin](https://www.linkedin.com/in/eng-socd/) - [github](https://github.com/socd06)  $\newcommand{\shared}[1]{[\![ #1 ]\!]}$  

## Definition

Beaver triples are [tuples](https://en.wikipedia.org/wiki/Tuple) of three values such that `a` and `b` are random uniform values and

$ c = ab $

Beaver triples are named after Donald Beaver, the author of the [paper](https://link.springer.com/chapter/10.1007/3-540-46766-1_34) where the technique was introduced.

## Geometric explanation

Beaver triples are applied in **private multiplication** where we want to compute $ \shared{z} = \shared{xy} $ and both $\shared{x}$ and $\shared{y}$ are shared values. This is also known as preprocessed material and is used to compute the multiplication as shown in the geometric explanation below.



![Beaver Triples Trick](img/beaver-triples-trick.jpg)

### Private multiplication
This has been implemented in different ways depending on the author and purpose of the protocol, we present a simplified overview of the process in favour of learning:
#### Precomputation
First-off, in the precomputation phase:
- a crypto provider (or trusted third party) computes a Beaver multiplication triple $ (a, b, c) $, such that `a` and `b` are random and $ c = ab $ and then
- It secret shares the triple with the party

#### Online Phase
Assumming all parties have a secret share of a multiplicatiion Beaver triple $\shared{a}, \shared{b}, \shared{c}$. The process is as follows:

- Each party computes $ \shared{x-a} $ and publishes their share of $ x - a $, revealing (reconstructing) $ \delta = x - a $ 


- Each party computes $ \shared{y-b} $ and publishes their share of $ y - b $, revealing (reconstructing) $ \epsilon = y - b $ 


- All parties compute $ z_i = c_i + a_i(y - b) + b_i(x - a) $, adding their shares

or, simplified
- $ z_i =  c_i + a_i \epsilon + \delta b_i  + \delta \epsilon  $


- Finally, an arbitrary party adds $ (x - a)(y - b) $ or $ \delta \epsilon $ for short, to the computation, revealing $ z = xy $


**Note:** Values inside this type of square brackets $\shared{}$ are secret shares.

### [Quiz] Compute the expected z<sub>alice</sub> and z<sub>bob</sub> shares of a private multiplication

Let our inputs be 

$ x = 6 $ and $ y = 4 $

and we consume the following triples:

$( a, \;b,\; c ) = ( 12, \; 26, \; 312 )$

Then let our inputs and triples be secret shared between `Alice` and `Bob`. Therefore, 

- `Alice` holds:

$ x_{alice} = 2, \; y_{alice}= -5 $

$ a_{alice} = 15, \; b_{alice} = -20, \; c_{alice} = 117  $

- and `Bob` holds:

$ x_{bob} = 4, \; y_{bob}= 9 $

$ a_{bob} = -3, \; b_{bob} = 46, \; c_{bob} = 195 $


|                       |Alice                   | | | | | |                     |Bob                  | |
|:----------------------|:----------------------:|-|-|-|-|-|:--------------------|:-------------------:|-|
|x<sub>alice</sub> = 2  |a<sub>alice</sub> = 15  | | | | | |x<sub>bob</sub> = 4  |a<sub>bob</sub> = -3 | |
|y<sub>alice</sub> = -5 |b<sub>alice</sub> = -20 | | | | | |y<sub>bob</sub> = 9 |b<sub>bob</sub> = 46 | |
|                       |c<sub>alice</sub> = 117 | | | | | |                     |c<sub>bob</sub> = 195| |


Compute the expected $z_{alice} $ and $ z_{bob} $ shares assuming Bob adds $ \delta \epsilon $. 

Fill the ____ spaces below with your answers. Feel free to implement the equation in a new cell or use whatever tool you'd like (e.g. a calculator), it's your call. 

In [2]:
# Run this cell to import the quizzes
from quiz import q3, q4

In [7]:
# Fill the ____ space below with your answer
x_A = 2
y_A = -5
a_A = 15
b_A = -20
c_A = 117
x_B = 4
y_B = 9
a_B = -3
b_B = 46
c_B = 195

delta = (x_A-a_A)+(x_B-a_B)
eps = (y_A - b_A)+(y_B-b_B)
z_alice = c_A + a_A * eps + delta * b_A 
# run to check your answer
q3.check(z_alice)

Success: Excellent!


True

In [4]:
# Uncomment the line below to see a hint
q3.hint

Hint: Reconstruct x - a and y - b by adding the Alice and Bob's [x-a] and [y-b]


In [8]:
# Uncomment the line below to see the solution
q3.solution

Solution: z_alice = -93


In [9]:
# Fill the ____ space below with your answer
z_bob = c_B + a_B * eps + b_B * delta + eps * delta

# run to check your answer
q4.check(z_bob)

Success: Correct! Bob's share is -15 + (6)(22) = 117


True

In [ ]:
# Uncomment the line below to see a hint
q4.hint

In [10]:
# Uncomment the line below to see the solution
q4.solution

Solution: z_bob = 117


## Implement Private multiplication with Beaver Triples
Now that you are aware of the theory, why don't you try implementing private multiplication from scratch? 

How did it go? There are many different ways to implement this so there is no right solution. We hope you were able to see how the math allows us to hide our inputs.

# (Solution:)
## Implementation
There are many ways to implement this, particularly on a production-grade application. In this lesson, we want you to understand how these principles work and not worry too much about the best possible implementation. Therefore, we implement private multiplication in a simplified (and not very secure) way. 

In [11]:
def beaver_triple(r):
    '''
    r = randomness
    '''
    a = randint(r)
    b = randint(r)
    c = a * b
    
    return (a, b, c)

In [12]:
# Define secret sharing from previous lesson

def n_share(s, r, n):
    '''
    s = secret
    r = randomness
    n = number of nodes, workers or participants
    '''
    share_lst = list()

    for i in range(n - 1):
        share_lst.append(randint(0,r))

    final_share = r - (sum(share_lst) % r) + s

    share_lst.append(final_share)

    return tuple(share_lst)

In [13]:
# also reusing the decryption function
def decrypt(shares, r):
    '''
    shares = iterable made of additive secret shares
    r = randomness
    '''
    return sum(shares) % r

In [14]:
# Import numpy randomness function
from numpy.random import randint

# Small Q in favour of computation speed
Q = 64601

We will use lists and dictionaries for simplicity.

In [15]:
# 0 - Create a dictionary per party

alice = dict(name="alice")
bob = dict(name="bob")

# and put them in a list
parties = [alice, bob]
parties

[{'name': 'alice'}, {'name': 'bob'}]

Enter integers to multiply together

In [16]:
x = int(input("Alice's input is: "))
y = int(input("Bob's input is: "))

Alice's input is: 15
Bob's input is: 98


In [17]:
# 1 - Secret share the inputs

x1, x2 = n_share(x,Q,len(parties))         

xsecrets = [ x1, x2 ] 

y1, y2 = n_share(y,Q,len(parties))         

ysecrets = [ y1, y2 ] 

for i, party in enumerate(parties):   
    party["x"] = xsecrets[i]
    party["y"] = ysecrets[i]
    
    print(party)

{'name': 'alice', 'x': 40206, 'y': 43555}
{'name': 'bob', 'x': 24410, 'y': 21144}


In [18]:
# 2 - Generate Beaver Triple

# Compute triples using Q
a,b,c = beaver_triple(Q)

triple = (a,b,c)
print("Triple (a,b,c) = ",triple)

Triple (a,b,c) =  (53759, 28838, 1550302042)


In [19]:
# 3 - Secret Share triple
for count, elem in enumerate(triple):
    
    # Additive secret share 
    shares = n_share(elem,Q,len(parties))         
    
    # a
    if count == 0:
        lit = "a"        
        
    # b
    elif count == 1:       
        lit = "b"
        
    # c
    else:
        lit = "c"
        
    print(lit,"=",elem,"split into", shares,"\n")        
        
    for party, share in enumerate(shares):
        
        parties[party][lit] = share

a = 53759 split into (60578, 57782) 

b = 28838 split into (43231, 50208) 

c = 1550302042 split into (30211, 1550336432) 



In this example, we can check that the triples have been split into shares among the parties correctly

In [20]:
alice

{'name': 'alice', 'x': 40206, 'y': 43555, 'a': 60578, 'b': 43231, 'c': 30211}

In [21]:
bob

{'name': 'bob',
 'x': 24410,
 'y': 21144,
 'a': 57782,
 'b': 50208,
 'c': 1550336432}

In [22]:
# Each party 
for party in parties:    
    
    # computes x - a    
    party["x-a"] = party["x"] - party["a"]
    print(f'{party["name"]} computes \n[x-a] = {party["x"]} - {party["a"]} = {party["x-a"]}')
    
    # and y - b    
    party["y-b"] = party["y"] - party["b"]
    print(f'{party["name"]} computes \n[y-b] = {party["y"]} - {party["b"]} = {party["y-b"]}')

alice computes 
[x-a] = 40206 - 60578 = -20372
alice computes 
[y-b] = 43555 - 43231 = 324
bob computes 
[x-a] = 24410 - 57782 = -33372
bob computes 
[y-b] = 21144 - 50208 = -29064


In [23]:
# revealing delta
delta = alice["x-a"] + bob["x-a"]
print("delta =",delta)
    
# and epsilon
epsilon = alice["y-b"] + bob["y-b"]

print("epsilon =",epsilon)

delta = -53744
epsilon = -28740


In [24]:
# and all parties compute using the reconstructed triples and the newly generated delta and epsilon variables

for i, party in enumerate(parties):
    party["z"] = party["c"] + delta * party["b"] + party["a"] * epsilon
    print(f'z_{party["name"]} = {party["z"]}')
    
    if i == len(parties)-1:
        party["z"] += delta * epsilon
        print(f'The last party ({party["name"]}) adds (delta)(epsilon) \n[z] = { party["z"] }')

z_alice = -4064388373
z_bob = -2808697000
The last party (bob) adds (delta)(epsilon) 
[z] = -1264094440


Since we introduced randomness to our parties' inputs using Q, we use the additive secret sharing decrypt function to remove that randomness. 

In [25]:
decrypt( [ alice["z"], bob["z"] ], Q )

1470

## Working with Matrices

Now that we know how to secret share, add and multiply integers privately, which is basic for deep learning networks we should learn how to do the same work with matrices. More importantly, neural networks programmed with [PyTorch](https://pytorch.org/) represent images as tensors. 

### Additive Secret Sharing

Borrowing from the previous lesson, we can use the same logic to secret share matrices by adding a `random_tensor` helper function.

In [26]:
# We use the secrets module to generate strong random numbers
from secrets import randbelow

# We use NumPy for math operations
import numpy as np

# and PyTorch to represent our data using tensors
import torch

def random_tensor(shape,r):
    '''
    shape = desired tensor shape
    r = randomness
    '''    
    values = [ randbelow(r) for _ in range(np.prod(shape)) ]
    return torch.tensor(values, dtype=torch.long).reshape(shape)

In [27]:
# Modifying secret sharing from previous lesson to generate random matrices

def matrix_share(m, r, n):
    '''
    m = matrix secret
    r = randomness
    n = number of nodes, workers or participants
    '''
    share_lst = list()

    for i in range(n - 1):
        
        # add the random_tensor helper function
        share_lst.append(random_tensor(m.shape,r))

    final_share = r - (sum(share_lst) % r) + m

    share_lst.append(final_share)
    
    # and return a tuple of random tensors    
    return tuple(share_lst)

Next, we do a quick test to verify our secret sharing function works

In [28]:
# We make an arbitrary tensor of 2x3 shape
test_tensor = torch.tensor([[1, 2, 3],
                            [3, 2, 1]])

# Make secret shares from our test tensor
n = 2
matrix_shares = matrix_share( test_tensor , Q, n)
matrix_shares

(tensor([[36823,  2932, 28802],
         [12043,   428, 36083]]),
 tensor([[27779, 61671, 35802],
         [52561, 64175, 28519]]))

In [29]:
# Can we decrypt the shares using our original function?
decrypt(matrix_shares, Q)

tensor([[1, 2, 3],
        [3, 2, 1]])

Success!

### Matrix Multiplication Refresher

For matrix multiplication, we need the columns of our first matrix to be the same as the rows in our second matrix.

In [30]:
x = torch.tensor([   # 4 x 3
        [1, 1, 1], 
        [2, 2, 2],
        [3, 3, 3],
        [4, 4, 4]
    ], dtype=torch.long)

y = torch.tensor([   # 3 x 2
        [0, 1], 
        [2, 3],
        [0, 2]  
    ], dtype=torch.long)

print(x.shape, y.shape)

torch.Size([4, 3]) torch.Size([3, 2])


In regular PyTorch, we can use the `torch.matmul` operation to do n-dimensional matrix multiplication, `x @ y` in short.

In [31]:
torch.matmul(x, y)

tensor([[ 2,  6],
        [ 4, 12],
        [ 6, 18],
        [ 8, 24]])

## Adapt your code to MatMul
Now that you know how to multiply and secret share matrices with PyTorch, try implementing private multiplication on your own.

# (Solution)

## Private Matrix Multiplication
### Matrix Beaver Triples
Adapting Beaver's principles to matrices, we can implement private matrix multiplication this way.

Following the same logic as before, we can make `a` and `b` random tensors and `matmul` them together to make `c = ab`

In [32]:
def matrix_triple(x: torch.LongTensor, y: torch.LongTensor, r: int):
    '''
    x = x tensor
    y = y tensor
    r = randomness
    '''
    # Generate random tensors with the same shape as our inputs
    a = random_tensor(x.shape,r)
    b = random_tensor(y.shape,r)
    
    # And we matrix multiply them to make c = ab
    c = torch.matmul(a,b)
    
    return a, b, c

In [33]:
matrix_triple(x, y, Q)

(tensor([[54261, 22842, 36712],
         [33821, 23434,  4077],
         [29253, 22758, 13275],
         [54746, 45999,  4314]]),
 tensor([[45662,  3875],
         [51785, 49437],
         [55187, 19193]]),
 tensor([[5686563896, 2044114745],
         [2982861591, 1367812894],
         [3246880941, 1493229696],
         [5119946785, 2568991915]]))

In [34]:
# Define 2 parties that will virtually hold the shares
parties = ["alice", "bob"]

In [35]:
# and we view their shapes
print(f'Matrix 1 Shape: {x.shape} \nMatrix 2 Shape: {y.shape}')

Matrix 1 Shape: torch.Size([4, 3]) 
Matrix 2 Shape: torch.Size([3, 2])


Then we secret share `x` and `y` between our two parties of Alice and Bob

In [36]:
# 1 - Secret share inputs

# secret share matrix x
x_sh = matrix_share(x, Q, len(parties))

# secret share matrix y
y_sh = matrix_share(y, Q, len(parties))

In [37]:
# 2 - Generate Matrix Beaver Triple
a, b, c = matrix_triple(x, y, Q)

matrix_triple = (a, b, c)
print("Triple (a,b,c) = \n", matrix_triple)

Triple (a,b,c) = 
 (tensor([[60025, 22642, 55171],
        [36797, 44979, 56696],
        [22652, 41714, 48197],
        [10156,  1576, 31212]]), tensor([[20371, 63827],
        [42351, 18430],
        [24374, 17802]]), tensor([[3526418571, 5230661877],
        [4036405620, 4186907281],
        [3402827184, 3072601218],
        [1034394340, 1232908716]]))


In [38]:
# 3 - Secret Share triples
a_sh = matrix_share(a, Q, len(parties))
b_sh = matrix_share(b, Q, len(parties))
c_sh = matrix_share(c, Q, len(parties))

In [39]:
def sub(x, y):
    """Emulates x - y for shared values"""
    
    n_party = len(x)
    z = [
        x[party] - y[party]
        for party in range(n_party)
    ]
        
    return z

In [40]:
epsilon = decrypt(sub(x_sh, a_sh), Q)
delta = decrypt(sub(y_sh, b_sh), Q)

print("epsilon =",epsilon)
print("delta =",delta)

epsilon = tensor([[ 4577, 41960,  9431],
        [27806, 19624,  7907],
        [41952, 22890, 16407],
        [54449, 63029, 33393]])
delta = tensor([[44230,   775],
        [22252, 46174],
        [40227, 46801]])


In [41]:
z_sh = [0] * len(parties) # initialize the shares
for party in range(len(parties)):
    
    z_sh[party] = c_sh[party] + epsilon @ b_sh[party] + a_sh[party] @ delta 
    
    if party == 0: # only add the public value once
        z_sh[party] += epsilon @ delta

In [42]:
decrypt(z_sh, Q)

tensor([[ 2,  6],
        [ 4, 12],
        [ 6, 18],
        [ 8, 24]])

In [43]:
# Expected:
x @ y

tensor([[ 2,  6],
        [ 4, 12],
        [ 6, 18],
        [ 8, 24]])

## Private Multiplication with PySyft
As we mentioned in the intro video, Beaver Triples is the backbone of the [SPDZ protocol](https://link.springer.com/chapter/10.1007%2F978-3-642-32009-5_38) which is mostly implemented in [PySyft](https://github.com/OpenMined/PySyft) already. So, we can do private multiplication without having to implement the math from scratch.

In [44]:
import torch
import syft as sy
hook = sy.TorchHook(torch)

In [45]:
alice = sy.VirtualWorker(hook, id="alice")
bob = sy.VirtualWorker(hook, id="bob")
# charlie = sy.VirtualWorker(hook, id="charlie")
secure_worker = sy.VirtualWorker(hook, "secure_worker")

### Private Integer Multiplication with PySyft

In [58]:
x = torch.tensor([6])
y = torch.tensor([8])

# And we additive share with our parties
x = x.share(alice, bob,  crypto_provider=secure_worker)
y = y.share(alice, bob,  crypto_provider=secure_worker) # charlie, only 2 people in pysyft 0.2.x

In [59]:
# Compute z = x * y
scalar_mul = x * y
scalar_mul

d:\program files\python\python37\tutorial-env\lib\site-packages\syft\frameworks\torch\tensors\interpreters\additive_shared.py:122: UserWarning: Use dtype instead of field
  warnings.warn("Use dtype instead of field")


(Wrapper)>[AdditiveSharingTensor]
	-> [PointerTensor | me:91924633965 -> alice:70728262199]
	-> [PointerTensor | me:49333075572 -> bob:48338158278]
	*crypto provider: secure_worker*

If we try to look at the result, we can see that our inputs have been scrambled and replaced with pointers and random numbers, just like above.

In [60]:
decrypted_scalar_mul = scalar_mul.get()
decrypted_scalar_mul.item()

48

It works! It may seem like we still need to write a lot of lines but consider that we are simulating four-workers environment where all inputs are hidden. 

### Private Matrix Multiplication with PySyft

Now, lets try something more complicated, like tensor(matrix) multiplication.

In [65]:
# feel free to play with these values
matrix1 = torch.tensor(
    [
        # 3 x 3
        [ 1,   1,  1],
        [ 1,   1,  1],
        [ 1,   1,  1]
    ], dtype=torch.long)

matrix2 = torch.tensor(
    [
        # 3 x 3
        [ 0,  -1,  0],
        [-1,   5, -1],
        [ 0,  -1,  0]
    ], dtype=torch.long)


In [66]:
matrix1 = matrix1.share(alice, bob, crypto_provider=secure_worker)
matrix2 = matrix2.share(alice, bob, crypto_provider=secure_worker)

In [67]:
tensor_mul = matrix1 * matrix2
tensor_mul

(Wrapper)>[AdditiveSharingTensor]
	-> [PointerTensor | me:98399545474 -> alice:65720889961]
	-> [PointerTensor | me:46059124347 -> bob:79206153246]
	*crypto provider: secure_worker*

So far so good...

In [68]:
decrypted_tensor_mul = tensor_mul.get()
decrypted_tensor_mul

tensor([[ 0, -1,  0],
        [-1,  5, -1],
        [ 0, -1,  0]])

It also works! Since it works for tensor multiplication it will also work for convolution operations.

All these operations, like we established on the previous lesson, work over a finite field of integers, but in neural networks and in real life, we use floats! FixedPrecision encoding is how we **fix** that problem, and our next lesson.